# Neighborhood Segmentation and Clustering Assignment
by Eylul Tekin

## Part 1

1. Start by creating a new Notebook for this assignment.

In [1]:
#import the necessary libraries
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.


In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head(5)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


3. To create the above dataframe:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

In [3]:
 #rename the Postal code column to PostalCode
df.rename(columns = {'Postal code':'PostalCode'}, inplace = True)
# drop Not assigned boroughs
df = df[df['Borough'] != 'Not assigned'] 
#df.sort_values(by = 'PostalCode', ascending = True, inplace = True)
df.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
#check the number of rows
print('The number of rows is: ', df.shape[0]) 

#check the number of unique postal codes in PostalCode
print('The number of unique postal codes is: ', df['PostalCode'].value_counts().sum()) 

#The numbers are the same. This shows that all postal codes are unique.

The number of rows is:  103
The number of unique postal codes is:  103


In [5]:
#replace "/" with ","
df['Neighborhood'] = df['Neighborhood'].str.replace('/',",")
df.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [6]:
#check a few examples
df[df['PostalCode'] == 'M5A']
df[df['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood
143,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [7]:
#check whether there are any 'Not assigned' neighborhoods
df[df['Neighborhood'] == 'Not assigned']

#there are none, so our dataset is ready

,PostalCode,Borough,Neighborhood


In [8]:
#reset the index values
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [9]:
#Last cell to print the number of rows
print('The number of rows is: ', df.shape[0]) 

The number of rows is:  103


## Part 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [122]:
!conda install -c conda-forge geocoder -y

Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [10]:
#create new empty columns for latitude and longitude
df["Latitude"] = ""
df["Longitude"] = ""
df.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",,
3,M6A,North York,"Lawrence Manor , Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",,


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None


# create a loop for each postal code
for i, code in enumerate(df['PostalCode']):    
    # loop until you get the coordinates
   
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
                            
    df.loc[i,'Latitude'] = lat_lng_coords[0]
    df.loc[i,'Longitude'] = lat_lng_coords[1]                          

__Important Note!:__ I left the above cell running the whole night, but the geocoder was not able to retrieve coordinates for all postal codes. Thus, I used the geospatial coordinates provided in the dataset.

In [12]:
geo_coor = pd.read_csv('Geospatial_Coordinates.csv')
#sort values based on postal code to match with our dataset
geo_coor.sort_values('Postal Code', ascending = True, inplace = True)
geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#sort values based on postal code to match with geospatial coordinates
df.sort_values('PostalCode', ascending = True, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern , Rouge",,
12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",,
18,M1E,Scarborough,"Guildwood , Morningside , West Hill",,
22,M1G,Scarborough,Woburn,,
26,M1H,Scarborough,Cedarbrae,,


In [14]:
#add geospatial coordinates to our dataset
df['Latitude'] = geo_coor['Latitude']
df['Longitude'] = geo_coor['Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern , Rouge",43.727929,-79.262029
12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.794200,-79.262029
18,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.778517,-79.346556
22,M1G,Scarborough,Woburn,43.770120,-79.408493
26,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


In [15]:
#reset the index values
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)
df.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df['Borough'].unique()),df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


## Part 3
#### Start with creating a map of Toronto using postal codes:

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label, code in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['PostalCode']):
    label_text = folium.Popup(str(label) + ' , ' + str(code), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label_text,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Get the data from Foursquare and create the relevant dataset:

In [29]:
CLIENT_ID = 'MNKLNBNALPNKN3NN3XA2W3CUNUMR13U0ZFVANYXSBV2VIO40' # your Foursquare ID
CLIENT_SECRET = 'SFGP30TCMTHFM5L0XWY2UHC1CFYMID3AY13ICBHBBE1IYJAF' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MNKLNBNALPNKN3NN3XA2W3CUNUMR13U0ZFVANYXSBV2VIO40
CLIENT_SECRET:SFGP30TCMTHFM5L0XWY2UHC1CFYMID3AY13ICBHBBE1IYJAF


In [30]:
num_ven = 100 
r = 500

def getNearbyVenues(codes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, name, lat, lng in zip(codes, names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            r, 
            num_ven)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
#create a dataset for venues in Toronto
toronto_venues = getNearbyVenues(codes=df['PostalCode'],
                                names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

In [73]:
print(toronto_venues.shape)
toronto_venues.head()

(2128, 8)


,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,"Malvern , Rouge",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
1,M1B,"Malvern , Rouge",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
2,M1B,"Malvern , Rouge",43.727929,-79.262029,Bros. CONVENIENCE,43.727781,-79.265708,Convenience Store
3,M1B,"Malvern , Rouge",43.727929,-79.262029,Tandy Leather,43.726974,-79.266513,Hobby Shop
4,M1B,"Malvern , Rouge",43.727929,-79.262029,Dollarama,43.726596,-79.266830,Discount Store


In [74]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


#### Prepare the data for cluster analysis:

In [75]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add the postal code and neighborhood columns back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 
toronto_onehot['Neighborhood_Name'] = toronto_venues['Neighborhood'] 

# move the postal code and neighborhood columns to the first and second columns
fixed_columns = [toronto_onehot.columns[-2]] + [toronto_onehot.columns[-1]]+ list(toronto_onehot.columns[:-2])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,PostalCode,Neighborhood_Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [76]:
#group based on postal code and reset the index
toronto_grouped = toronto_onehot.groupby(['PostalCode']).mean().reset_index()
toronto_grouped.head()

,PostalCode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0

#### Cluster analysis:

In [107]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1], dtype=int32)

In [108]:
#the function to get most common venues for each postal code
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [109]:
#create a dataset with top 10 venues for each postal code
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Hobby Shop,Train Station,Department Store,Discount Store,Convenience Store,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Skating Rink,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Restaurant
3,M1G,Ramen Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Pizza Place,Café,Sandwich Place,Lounge,Japanese Restaurant,Plaza
4,M1H,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Athletics & Sports,Baseball Field,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [110]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('PostalCode'), on='PostalCode')

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(103, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.727929,-79.262029,1.0,Hobby Shop,Train Station,Department Store,Discount Store,Convenience Store,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.794200,-79.262029,1.0,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Skating Rink,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Restaurant
3,M1G,Scarborough,Woburn,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Pizza Place,Café,Sandwich Place,Lounge,Japanese Restaurant,Plaza
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,1.0,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Athletics & Sports,Baseball Field,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [111]:
#sometimes clusters can result in NaN, make sure to exclude them
toronto_merged=toronto_merged.dropna()
toronto_merged.shape

(100, 16)

In [126]:
# create map
map_toronto_cluster = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.9).add_to(map_toronto_cluster)
       
map_toronto_cluster

In [125]:
#Cluster0
print('{} postal codes belong to this cluster'.format(len(toronto_merged[toronto_merged['Cluster Labels'] == 0.0])))
toronto_merged[toronto_merged['Cluster Labels'] == 0.0]

9 postal codes belong to this cluster


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M1W,Scarborough,"Steeles West , L'Amoreaux West",43.653654,-79.506944,0.0,Park,River,Comfort Food Restaurant,Comic Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
30,M3K,North York,Downsview,43.685347,-79.338106,0.0,Park,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
36,M4C,East York,Woodbine Heights,43.815252,-79.284577,0.0,Park,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
38,M4G,East York,Leaside,43.752758,-79.400049,0.0,Park,Bank,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
47,M4S,Central Toronto,Davisville,43.713756,-79.490074,0.0,Park,Basketball Court,Bakery,Construction & Landscaping,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
58,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.737473,-79.464763,0.0,Park,Airport,Business Service,Playground,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
65,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.689026,-79.453512,0.0,Park,Pool,Women's Store,Golf Course,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
75,M6G,Downtown Toronto,Christie,43.753259,-79.329656,0.0,Park,Construction & Landscaping,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
96,M9L,North York,Humber Summit,43.679563,-79.377529,0.0,Park,Playground,Trail,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


In [127]:
#Cluster1
print('{} postal codes belong to this cluster'.format(len(toronto_merged[toronto_merged['Cluster Labels'] == 1.0])))
toronto_merged[toronto_merged['Cluster Labels'] == 1.0]

89 postal codes belong to this cluster


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.727929,-79.262029,1.0,Hobby Shop,Train Station,Department Store,Discount Store,Convenience Store,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.794200,-79.262029,1.0,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Skating Rink,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Restaurant
3,M1G,Scarborough,Woburn,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Pizza Place,Café,Sandwich Place,Lounge,Japanese Restaurant,Plaza
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,1.0,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Athletics & Sports,Baseball Field,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697,1.0,Business Service,Food Truck,Baseball Field,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Diner
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.709060,-79.363452,1.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Furniture / Home Store,Bank,Shopping Mall,Juice Bar,Bike Shop,Restaurant,Beer Store
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.728020,-79.388790,1.0,Park,Bus Line,Swim School,Dim Sum Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.667967,-79.367675,1.0,Coffee Shop,Restaurant,Café,Bakery,Italian Restaurant,Pizza Place,Pub,General Entertainment,Indian Restaurant,Japanese Restaurant
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.650571,-79.384568,1.0,Coffee Shop,Café,Restaurant,Gym,American Restaurant,Deli / Bodega,Hotel,Clothing Store,Thai Restaurant,Pizza Place


In [133]:
#Cluster2
print('{} postal code belongs to this cluster'.format(len(toronto_merged[toronto_merged['Cluster Labels'] == 2.0])))
toronto_merged[toronto_merged['Cluster Labels'] == 2.0]

1 postal code belongs to this cluster


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M3A,North York,Parkwoods,43.806686,-79.194353,2.0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [132]:
#Cluster3
print('{} postal code belongs to this cluster'.format(len(toronto_merged[toronto_merged['Cluster Labels'] == 3.0])))
toronto_merged[toronto_merged['Cluster Labels'] == 3.0]

1 postal code belong to this cluster


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.706876,-79.518188,3.0,Convenience Store,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
